## Extraction of images from CZI files and selection of best layers

In [1]:
from skimage import exposure
import numpy as np
import czifile, tifffile
import os
from skimage import feature, color, io
import matplotlib.pyplot as plt

def process_images(image_dir, output_dir):
    # Ensure the output directory exists
    os.makedirs(output_dir, exist_ok=True)
    
    # List all CZI files in the directory
    image_paths = [os.path.join(image_dir, filename) for filename in os.listdir(image_dir) if filename.endswith('.czi')]
    print(f"Number of images found: {len(image_paths)}")    
    
    # Define the function to calculate LBP entropy
    def calculate_lbp(image, points=24, radius=8):
        lbp = feature.local_binary_pattern(image, points, radius, method="uniform")
        (hist, _) = np.histogram(lbp.ravel(), bins=np.arange(0, points + 3), range=(0, points + 2))
        hist = hist.astype("float")
        hist /= (hist.sum() + 1e-7)
        return -1 * (hist * np.log2(hist + 1e-7)).sum()

    for image_file in image_paths:
        lbp_entropies = []

        try:
            print(f"Computing LBP for {image_file}")
            with czifile.CziFile(image_file) as czi:
                image_array = czi.asarray()

            for stack_index in range(image_array.shape[4]):
                stack = image_array[0, 0, 0, 0, stack_index, :, :, 0]
                contrast_img = exposure.rescale_intensity(stack, in_range='image')
                contrast_img = exposure.equalize_hist(contrast_img)
                lbp_entropy = calculate_lbp(contrast_img)
                lbp_entropies.append((stack_index, lbp_entropy, contrast_img))
                
        except Exception as e:
            print(f"Error processing file {image_file}: {e}")

        lbp_entropies.sort(key=lambda x: x[1], reverse=True)

        try:
            top_stack_index, _, top_stack = lbp_entropies[0]
            filename = os.path.basename(image_file).replace(".czi", f"_stack_{top_stack_index}.tif")
            tifffile.imsave(os.path.join(output_dir, filename), top_stack)
            print(f"Saved {filename} to {output_dir}")

        except Exception as e:
            print(f"Error saving top stack: {e}")

# Define the directories that contain your images
image_dirs = [r'E:\Akita\1 Traditional Area Fraction Vessel Density\FG12_NG004_randomizer\Original Stack czi',
              r'E:\Akita\1 Traditional Area Fraction Vessel Density\PBS_11C7_randamizer\Original Stack Czi']
output_dirs = [r'data/FG12', r'data/PBS']

# Process images for each directory
for image_dir, output_dir in zip(image_dirs, output_dirs):
    process_images(image_dir, output_dir)

print("Done!")


Number of images found: 28
Computing LBP for E:\Akita\1 Traditional Area Fraction Vessel Density\FG12_NG004_randomizer\Original Stack czi\Akita_12_OD_25x.czi


c:\Users\alois\anaconda3\Lib\site-packages\skimage\feature\texture.py:353: UserWarning: Applying `local_binary_pattern` to floating-point images may give unexpected results when small numerical differences between adjacent pixels are present. It is recommended to use this function with images of integer dtype.
  warnings.warn(
C:\Users\alois\AppData\Local\Temp\ipykernel_9044\3414102960.py:47: DeprecationWarning: <tifffile.imsave> is deprecated. Use tifffile.imwrite
  tifffile.imsave(os.path.join(output_dir, filename), top_stack)


Saved Akita_12_OD_25x_stack_6.tif to data/FG12
Computing LBP for E:\Akita\1 Traditional Area Fraction Vessel Density\FG12_NG004_randomizer\Original Stack czi\Akita_12_OE_25x.czi
Saved Akita_12_OE_25x_stack_8.tif to data/FG12
Computing LBP for E:\Akita\1 Traditional Area Fraction Vessel Density\FG12_NG004_randomizer\Original Stack czi\Akita_14_OD_25x.czi
Saved Akita_14_OD_25x_stack_5.tif to data/FG12
Computing LBP for E:\Akita\1 Traditional Area Fraction Vessel Density\FG12_NG004_randomizer\Original Stack czi\Akita_14_OE_25x.czi
Saved Akita_14_OE_25x_stack_7.tif to data/FG12
Computing LBP for E:\Akita\1 Traditional Area Fraction Vessel Density\FG12_NG004_randomizer\Original Stack czi\Akita_15_OD_25x.czi
Saved Akita_15_OD_25x_stack_5.tif to data/FG12
Computing LBP for E:\Akita\1 Traditional Area Fraction Vessel Density\FG12_NG004_randomizer\Original Stack czi\Akita_15_OE_25x.czi
Saved Akita_15_OE_25x_stack_4.tif to data/FG12
Computing LBP for E:\Akita\1 Traditional Area Fraction Vessel D

In [ ]:
"""
from skimage import exposure
import numpy as np
import czifile, tifffile
import os
from skimage import feature, color, io
import matplotlib.pyplot as plt

# Define the directory that contains your images
image_dir = r'E:\Akita\1 Traditional Area Fraction Vessel Density\FG12_NG004_randomizer\Original Stack czi'
image_paths = [os.path.join(image_dir, filename) for filename in os.listdir(image_dir) if filename.endswith('.czi')]

# Define the function to calculate LBP entropy
def calculate_lbp(image, points=24, radius=8):
    lbp = feature.local_binary_pattern(image, points, radius, method="uniform")
    (hist, _) = np.histogram(lbp.ravel(), bins=np.arange(0, points + 3), range=(0, points + 2))
    hist = hist.astype("float")
    hist /= (hist.sum() + 1e-7)
    return -1 * (hist * np.log2(hist + 1e-7)).sum()

# Iterate over all image files
for image_file in image_paths:
    # Initialize a list to store the LBP entropies
    lbp_entropies = []

    try:
        print(f"Computing LBP for {image_file}")  # Print the image file being processed
        # Process the CZI file
        with czifile.CziFile(image_file) as czi:
            image_array = czi.asarray()

        # Iterate over the stacks, given your image array shape (1, 1, 1, 1, 18, 8806, 8346, 1)
        for stack_index in range(image_array.shape[4]):
            # Extract the stack
            stack = image_array[0, 0, 0, 0, stack_index, :, :, 0]

            # Enhance the stack
            contrast_img = exposure.rescale_intensity(stack, in_range='image')
            contrast_img = exposure.equalize_hist(contrast_img)

            # Calculate the LBP entropy
            lbp_entropy = calculate_lbp(contrast_img)

            # Append the LBP entropy to the list
            lbp_entropies.append((stack_index, lbp_entropy, contrast_img))
            
    except Exception as e:
        print(f"Error processing file {image_file}: {e}")

    # Sort the images by their LBP entropy in descending order
    lbp_entropies.sort(key=lambda x: x[1], reverse=True)

    try:
        top_stack_index, _, top_stack = lbp_entropies[0]
        print(f"Stack with highest LBP entropy: {top_stack_index}")  # Print the stack with the highest LBP entropy

        # Modify the filename to the desired format
        filename = os.path.basename(image_file).replace(".czi", "")
        filename = filename.rsplit('_', 1)[0] + f"_stack_{top_stack_index}.tif"

        # Save the top stack
        tifffile.imsave(f'data/FG12/{filename}', top_stack)

    except Exception as e:
        print(f"Error saving top stack: {e}")
"""
